In [1]:
import numpy as np
import nibabel as nib
import scipy.ndimage as ndi
from skimage import filters, measure, morphology
import matplotlib.pyplot as plt

TASK1.1


Loading the ct images

In [2]:
def load_ct_image(filepath):
    ct_img = nib.load(filepath)
    ct_data = ct_img.get_fdata()
    return ct_data, ct_img.affine, ct_img.header

Segmenting the bones

In [3]:
def segment_bones(ct_data, femur_threshold=300, tibia_threshold=300):

    
    
    depth, height, width = ct_data.shape
    print(f"CT volume dimensions: {depth}x{height}x{width}")  #to get the image dimension 
    
    segmented_femur = np.zeros_like(ct_data, dtype=bool)
    segmented_tibia = np.zeros_like(ct_data, dtype=bool)
    

    print(f"Using threshold value: {femur_threshold} HU for bones")  #creating the initial bone mask using threshold
    bone_mask = ct_data > femur_threshold
    
    # Cleaning up the mask
    bone_mask = morphology.remove_small_objects(bone_mask, min_size=50)
    bone_mask = morphology.binary_closing(bone_mask, morphology.ball(1))
    
    # Finding a good slice to analyze (where both bones are visible)
    # Calculate sum across each slice to find slices with significant bone content
    slice_bone_content = np.sum(bone_mask, axis=(1, 2))
    significant_slices = np.where(slice_bone_content > 100)[0]
    
    if len(significant_slices) == 0:
        print("No significant bone content found. Adjusting threshold...")
        # If no significant bone content is found then go with the lower threshold
        bone_mask = ct_data > (femur_threshold - 100)
        bone_mask = morphology.remove_small_objects(bone_mask, min_size=50)
        slice_bone_content = np.sum(bone_mask, axis=(1, 2))
        significant_slices = np.where(slice_bone_content > 100)[0]
    
    print(f"Found {len(significant_slices)} slices with significant bone content")
    
    # If we still can't find significant slices, use anatomical approach
    if len(significant_slices) == 0:
        print("Still no significant bone content found. Using anatomical approach...")
        # Divide the volume into anatomical regions
        femur_region = slice(0, depth // 2)  # Upper half for femur
        tibia_region = slice(depth // 2, depth)  # Lower half for tibia
        
        # Apply different thresholds for each region
        segmented_femur[femur_region] = ct_data[femur_region] > femur_threshold
        segmented_tibia[tibia_region] = ct_data[tibia_region] > tibia_threshold
    else:
        # Analyze each significant slice to identify femur and tibia
        for slice_idx in significant_slices:
            slice_mask = bone_mask[slice_idx]
            labeled_slice, num_labels = measure.label(slice_mask, return_num=True)
            
            if num_labels < 1:
                continue
                
            # Get region properties
            regions = measure.regionprops(labeled_slice)
            
            # For simplicity with this particular CT orientation:
            # Based on the image shown, the femur appears on the right side (higher x values)
            # and tibia appears on the left side (lower x values)
            # This is specific to the current dataset orientation
            
            # Sort regions by centroid x-coordinate (horizontal position)
            regions.sort(key=lambda r: r.centroid[1], reverse=True)  # Sort right to left
            
            # Apply to specific regions (may need adjustment based on actual image orientation)
            for i, region in enumerate(regions):
                if i == 0:  # Right-most region (assumed to be femur in this orientation)
                    segmented_femur[slice_idx][labeled_slice == region.label] = True
                elif i == 1:  # Second right-most region (assumed to be tibia in this orientation)
                    segmented_tibia[slice_idx][labeled_slice == region.label] = True
    
    # Clean up the segmentation masks
    segmented_femur = morphology.binary_closing(segmented_femur, morphology.ball(2))
    segmented_tibia = morphology.binary_closing(segmented_tibia, morphology.ball(2))
    
    # Remove small disconnected components
    segmented_femur = morphology.remove_small_objects(segmented_femur, min_size=100)
    segmented_tibia = morphology.remove_small_objects(segmented_tibia, min_size=100)
    
    # If the femur or tibia mask is empty, try a different approach
    if not np.any(segmented_femur) or not np.any(segmented_tibia):
        print("Warning: One or both segmentation masks are empty. Trying alternative approach...")
        # Use anatomical knowledge about the rough positions of femur and tibia in this CT
        # For this specific orientation, divide the image into regions
        
        # For this specific dataset, create spatial masks based on the approximate locations
        # These values may need adjustment based on the actual data
        height_mid = height // 2
        width_mid = width // 2
        
        # Create spatial masks for femur (right upper quadrant) and tibia (right lower quadrant)
        femur_spatial_mask = np.zeros((depth, height, width), dtype=bool)
        tibia_spatial_mask = np.zeros((depth, height, width), dtype=bool)
        
        # Define spatial regions (adjust based on CT orientation)
        femur_spatial_mask[:, :height_mid, width_mid:] = True  # Upper right quadrant
        tibia_spatial_mask[:, height_mid:, width_mid:] = True  # Lower right quadrant
        
        # Combine spatial masks with intensity threshold
        segmented_femur = (ct_data > femur_threshold) & femur_spatial_mask
        segmented_tibia = (ct_data > tibia_threshold) & tibia_spatial_mask
        
        # Clean up
        segmented_femur = morphology.remove_small_objects(segmented_femur, min_size=100)
        segmented_tibia = morphology.remove_small_objects(segmented_tibia, min_size=100)
    
    print("Bone segmentation completed")
    return segmented_femur, segmented_tibia

In [4]:
#function to save the segmentation 
def save_segmentation(segmentation, affine, header, output_path):
    print(f"Saving segmentation to {output_path}")
    # Convert boolean to int (0 and 1)
    segmentation_int = segmentation.astype(np.int16)
    # Create NIfTI image
    nii_img = nib.Nifti1Image(segmentation_int, affine, header)
    # Save as nii.gz
    nib.save(nii_img, output_path)


In [5]:
#function to visualize the segments 
def visualize_segmentation(ct_data, femur_mask, tibia_mask, slice_num=None):
    if slice_num is None:
        # Find a slice with both bones visible
        combined_mask = femur_mask | tibia_mask
        slice_indices = np.where(combined_mask.sum(axis=(1, 2)) > 0)[0]
        if len(slice_indices) > 0:
            slice_num = slice_indices[len(slice_indices) // 2]  # Middle slice with bones
        else:
            slice_num = ct_data.shape[0] // 2  # Default to middle slice
    
    plt.figure(figsize=(15, 5))
    
    # Original CT slice
    plt.subplot(131)
    plt.imshow(ct_data[slice_num], cmap='gray')
    plt.title('Original CT Slice')
    plt.axis('off')
    
    # Get the min/max values for consistent display
    vmin = np.min(ct_data[slice_num])
    vmax = np.max(ct_data[slice_num])
    
    # Femur segmentation with clear highlight
    plt.subplot(132)
    plt.imshow(ct_data[slice_num], cmap='gray', vmin=vmin, vmax=vmax)
    
    # Create a highlighted overlay for femur
    femur_overlay = np.zeros_like(ct_data[slice_num], dtype=float)
    femur_overlay[femur_mask[slice_num]] = 1.0
    
    # Apply the overlay with high contrast
    plt.imshow(femur_overlay, alpha=0.7, cmap='Reds')
    plt.title('Femur Segmentation')
    plt.axis('off')
    
    # Tibia segmentation with clear highlight
    plt.subplot(133)
    plt.imshow(ct_data[slice_num], cmap='gray', vmin=vmin, vmax=vmax)
    
    # Create a highlighted overlay for tibia
    tibia_overlay = np.zeros_like(ct_data[slice_num], dtype=float)
    tibia_overlay[tibia_mask[slice_num]] = 1.0
    
    # Apply the overlay with high contrast
    plt.imshow(tibia_overlay, alpha=0.7, cmap='Blues')
    plt.title('Tibia Segmentation')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('segmentation_visualization.png')
    plt.close()
    
    # Print the segmentation statistics
    femur_voxels = np.sum(femur_mask)
    tibia_voxels = np.sum(tibia_mask)
    print(f"Femur segmentation: {femur_voxels} voxels")
    print(f"Tibia segmentation: {tibia_voxels} voxels")

In [6]:
def main():
    # Input and output paths
    input_path = "./3702_left_knee.nii"  # Replace with actual path
    output_femur_path = "segmented_femur.nii.gz"
    output_tibia_path = "segmented_tibia.nii.gz"
    
    # Step 1: Load the CT image
    ct_data, affine, header = load_ct_image(input_path)
    
    # Step 2: Segment the bones
    # Using a higher threshold of 300 HU which is more appropriate for dense bone
    femur_mask, tibia_mask = segment_bones(ct_data, femur_threshold=300, tibia_threshold=300)
    
    # Step 3: Save the segmentations
    save_segmentation(femur_mask, affine, header, output_femur_path)
    save_segmentation(tibia_mask, affine, header, output_tibia_path)
    
    # Step 4: Visualize the results
    visualize_segmentation(ct_data, femur_mask, tibia_mask)
    
    print("Bone segmentation task completed successfully")
    
    # Optional: Display a cross-section where both bones are visible
    # Find a good slice to show
    combined_mask = femur_mask | tibia_mask
    slice_indices = np.where(combined_mask.sum(axis=(1, 2)) > 0)[0]
    if len(slice_indices) > 0:
        good_slice = slice_indices[len(slice_indices) // 2]
        print(f"A good slice for visualization is: {good_slice}")
    else:
        print("No slices with both bones found")

if __name__ == "__main__":
    main()

CT volume dimensions: 512x512x216
Using threshold value: 300 HU for bones
Found 183 slices with significant bone content
Bone segmentation completed
Saving segmentation to segmented_femur.nii.gz
Saving segmentation to segmented_tibia.nii.gz
Femur segmentation: 75612 voxels
Tibia segmentation: 36776 voxels
Bone segmentation task completed successfully
A good slice for visualization is: 403


TASK 1.2 & 1.3


Loading the data and saving the data 

In [16]:
def load_nii(file_path):
    
    nii_img = nib.load(file_path)
    return nii_img.get_fdata(), nii_img.affine, nii_img.header

def save_nii(data, affine, header, output_path):
    
    nii_img = nib.Nifti1Image(data, affine, header)
    nib.save(nii_img, output_path)
    print(f"Saved: {output_path}")

For contour expansion and randomized contour adjustment

In [17]:
import numpy as np
import nibabel as nib
import scipy.ndimage as ndimage
def expand_mask(mask, expansion_mm, voxel_spacing):

    # Convert expansion from mm to voxels in each dimension
    expansion_voxels = [expansion_mm / spacing for spacing in voxel_spacing]
    
    # Calculate structure element size for dilation
    # Using spherical structuring element for uniform expansion
    radius = max(expansion_voxels)
    struct_size = int(2 * radius + 1)
    
    # Create spherical structuring element
    struct = ndimage.generate_binary_structure(3, 1)
    struct = ndimage.iterate_structure(struct, int(radius))
    
    # Dilate the mask
    expanded_mask = ndimage.binary_dilation(mask, structure=struct).astype(np.int16)
    
    return expanded_mask


In [18]:
def randomize_mask(original_mask, expanded_mask, max_expansion_mm, voxel_spacing):

    # Create a mask of the expansion region (difference between expanded and original)
    expansion_region = expanded_mask.astype(int) - original_mask.astype(int)
    
    # Generate random values between 0 and max_expansion_mm for each voxel
    random_values = np.random.uniform(0, max_expansion_mm, size=expansion_region.shape)
    
    # Create distance map from the original mask boundary
    # (distance of each voxel in the expansion region from the original mask)
    distance_map = ndimage.distance_transform_edt(~original_mask.astype(bool), sampling=voxel_spacing)
    
    # Only consider distances in the expansion region
    distance_map = distance_map * expansion_region
    
    # Where the random value is less than the distance, keep the voxel in the randomized mask
    # This ensures we don't exceed the max expansion and don't go below the original mask
    random_mask = np.logical_or(
        original_mask.astype(bool),
        np.logical_and(
            expansion_region.astype(bool),
            random_values > distance_map
        )
    ).astype(np.int16)
    
    return random_mask

In [19]:
import os 
def process_segmentation(input_file, expansion_mm=2.0, make_random=True):

    # Extract filename parts
    base_name = os.path.basename(input_file)
    base_dir = os.path.dirname(input_file)
    name_without_ext = os.path.splitext(base_name)[0]
    if name_without_ext.endswith('.nii'):
        name_without_ext = os.path.splitext(name_without_ext)[0]
    
    # Load the segmentation mask
    mask_data, affine, header = load_nii(input_file)
    
    # Get voxel spacing from the affine matrix
    voxel_spacing = np.array([abs(affine[i, i]) for i in range(3)])
    
    # Create output filenames
    expanded_output = os.path.join(base_dir, f"{name_without_ext}_expanded_{expansion_mm}mm.nii.gz")
    random_output = os.path.join(base_dir, f"{name_without_ext}_random_{expansion_mm}mm.nii.gz")
    
    # Expand the mask
    expanded_mask = expand_mask(mask_data, expansion_mm, voxel_spacing)
    
    # Save expanded mask
    save_nii(expanded_mask, affine, header, expanded_output)
    
    # Create and save randomized mask if requested
    if make_random:
        random_mask = randomize_mask(mask_data, expanded_mask, expansion_mm, voxel_spacing)
        save_nii(random_mask, affine, header, random_output)
        return expanded_output, random_output
    
    return expanded_output, None

In [20]:
def main():
    # Process femur segmentation
    femur_file = "segmented_femur.nii.gz"
    expansion_mm = 2.0  # 2mm expansion parameter
    
    print(f"Processing femur segmentation with {expansion_mm}mm expansion...")
    femur_expanded, femur_random = process_segmentation(femur_file, expansion_mm)
    
    # Process tibia segmentation
    tibia_file = "segmented_tibia.nii.gz"
    
    print(f"Processing tibia segmentation with {expansion_mm}mm expansion...")
    tibia_expanded, tibia_random = process_segmentation(tibia_file, expansion_mm)
    
    print("\nAll processing complete!")
    print(f"Expanded masks: {femur_expanded}, {tibia_expanded}")
    print(f"Randomized masks: {femur_random}, {tibia_random}")

if __name__ == "__main__":
    main()

Processing femur segmentation with 2.0mm expansion...
Saved: segmented_femur_expanded_2.0mm.nii.gz
Saved: segmented_femur_random_2.0mm.nii.gz
Processing tibia segmentation with 2.0mm expansion...
Saved: segmented_tibia_expanded_2.0mm.nii.gz
Saved: segmented_tibia_random_2.0mm.nii.gz

All processing complete!
Expanded masks: segmented_femur_expanded_2.0mm.nii.gz, segmented_tibia_expanded_2.0mm.nii.gz
Randomized masks: segmented_femur_random_2.0mm.nii.gz, segmented_tibia_random_2.0mm.nii.gz


TASK 1.4


In [21]:
import os
import numpy as np
import nibabel as nib
from scipy import ndimage
import matplotlib.pyplot as plt

def load_segmentation(file_path):
    """Load segmentation mask from NIfTI file."""
    print(f"Loading: {file_path}")
    return nib.load(file_path)

def save_segmentation(data, affine, output_path):
    """Save segmentation mask as NIfTI file."""
    nib_img = nib.Nifti1Image(data, affine)
    nib.save(nib_img, output_path)
    print(f"Saved: {output_path}")

In [22]:
def expand_mask(mask_data, distance_mm, voxel_spacing):
    """Expand segmentation mask uniformly by specified distance in mm."""
    # Convert distance from mm to voxels
    distance_voxels = [distance_mm / spacing for spacing in voxel_spacing]
    
    # Create structuring element for dilation
    radius = int(max(distance_voxels))
    struct = ndimage.generate_binary_structure(3, 1)
    struct = ndimage.iterate_structure(struct, radius)
    
    # Dilate the mask
    expanded_mask = ndimage.binary_dilation(mask_data, structure=struct).astype(mask_data.dtype)
    
    return expanded_mask

def randomize_mask(original_mask, expanded_mask, random_seed=42):
    """Create randomized mask between original and expanded mask."""
    np.random.seed(random_seed)
    
    # Find the boundary region (difference between expanded and original)
    boundary = expanded_mask.astype(int) - original_mask.astype(int)
    boundary_indices = np.where(boundary > 0)
    
    # Generate random values for boundary voxels
    num_boundary_voxels = len(boundary_indices[0])
    random_values = np.random.random(num_boundary_voxels)
    
    # Create a copy of the original mask for the randomized result
    randomized_mask = np.copy(original_mask)
    
    # Randomly include some boundary voxels (approx 50%)
    for i in range(num_boundary_voxels):
        if random_values[i] > 0.5:
            x, y, z = boundary_indices[0][i], boundary_indices[1][i], boundary_indices[2][i]
            randomized_mask[x, y, z] = 1
    
    return randomized_mask

In [23]:
def get_binary_mask(segmentation_data):
    """Extract binary mask from segmentation (any non-zero value)."""
    return (segmentation_data > 0).astype(np.int16)


In [24]:
def detect_landmarks(tibia_mask, voxel_spacing):
    """
    Detect medial and lateral lowest points on tibial surface.
    
    Returns:
        tuple: (medial_point, lateral_point) as (x, y, z) coordinates in mm
    """
    # Get indices of all tibia voxels
    tibia_indices = np.where(tibia_mask > 0)
    
    if len(tibia_indices[0]) == 0:
        print("Warning: Empty mask, cannot detect landmarks")
        return (0, 0, 0), (0, 0, 0)
    
    # Find the lowest points (in superior-inferior direction, typically z-axis in medical imaging)
    min_z = np.min(tibia_indices[2])
    
    # Get all points at the lowest z level
    lowest_points_indices = [(tibia_indices[0][i], tibia_indices[1][i], tibia_indices[2][i]) 
                             for i in range(len(tibia_indices[0])) 
                             if tibia_indices[2][i] == min_z]
    
    if not lowest_points_indices:
        print("Warning: Could not find lowest points")
        return (0, 0, 0), (0, 0, 0)
    
    # Sort points by x coordinate (lateral-medial axis)
    lowest_points_indices.sort(key=lambda p: p[0])
    
    # The most medial and lateral points will be at the extremes of the sorted list
    lateral_point_indices = lowest_points_indices[0]  # Smallest x (left/lateral)
    medial_point_indices = lowest_points_indices[-1]  # Largest x (right/medial)
    
    # Convert from voxel indices to mm coordinates
    lateral_point_mm = tuple(idx * spacing for idx, spacing in zip(lateral_point_indices, voxel_spacing))
    medial_point_mm = tuple(idx * spacing for idx, spacing in zip(medial_point_indices, voxel_spacing))
    
    return medial_point_mm, lateral_point_mm

In [25]:
def extract_surface_points(mask_data):
    """Extract the surface points of a 3D binary mask."""
    # Create a structure for erosion
    struct = ndimage.generate_binary_structure(3, 1)
    
    # Erode the mask
    eroded = ndimage.binary_erosion(mask_data, structure=struct)
    
    # The surface is the difference between the original and eroded mask
    surface = mask_data.astype(int) - eroded.astype(int)
    
    return surface

In [26]:
def main():
    # File paths 
    input_tibia_file = "segmented_tibia_random_2.0mm.nii.gz"  # Using your existing file
    output_dir = "tibia_landmarks"
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Parameters
    expansion_distance_1 = 2.0  # mm
    expansion_distance_2 = 4.0  # mm
    
    # Load tibia segmentation
    tibia_img = load_segmentation(input_tibia_file)
    tibia_data = tibia_img.get_fdata()
    voxel_spacing = tibia_img.header.get_zooms()[:3]  # Get voxel dimensions in mm
    
    # Convert to binary mask if needed
    original_mask = get_binary_mask(tibia_data)
    
    print(f"Loaded tibia mask with shape: {original_mask.shape}, voxel spacing: {voxel_spacing}")
    
    # Generate required masks
    print("Generating masks...")
    expanded_mask_2mm = expand_mask(original_mask, expansion_distance_1, voxel_spacing)
    expanded_mask_4mm = expand_mask(original_mask, expansion_distance_2, voxel_spacing)
    
    # We'll create additional randomized masks as needed
    randomized_mask_1 = randomize_mask(original_mask, expanded_mask_2mm, random_seed=42)
    randomized_mask_2 = randomize_mask(original_mask, expanded_mask_2mm, random_seed=123)
    
    # Save all masks
    print("Saving masks...")
    save_segmentation(original_mask, tibia_img.affine, os.path.join(output_dir, "original_mask.nii.gz"))
    save_segmentation(expanded_mask_2mm, tibia_img.affine, os.path.join(output_dir, "expanded_2mm_mask.nii.gz"))
    save_segmentation(expanded_mask_4mm, tibia_img.affine, os.path.join(output_dir, "expanded_4mm_mask.nii.gz"))
    save_segmentation(randomized_mask_1, tibia_img.affine, os.path.join(output_dir, "randomized_mask_1.nii.gz"))
    save_segmentation(randomized_mask_2, tibia_img.affine, os.path.join(output_dir, "randomized_mask_2.nii.gz"))
    
    # Detect landmarks for all masks
    print("Detecting landmarks...")
    masks = {
        "Original Mask": original_mask,
        "2mm Expanded Mask": expanded_mask_2mm, 
        "4mm Expanded Mask": expanded_mask_4mm,
        "Randomized Mask 1": randomized_mask_1,
        "Randomized Mask 2": randomized_mask_2
    }
    
    # Store landmark coordinates for each mask
    landmark_results = {}
    
    print("\nLandmark Coordinates (in mm):")
    print("-" * 80)
    print(f"{'Mask Type':<20} {'Medial Point':<30} {'Lateral Point':<30}")
    print("-" * 80)
    
    for mask_name, mask_data in masks.items():
        medial_point, lateral_point = detect_landmarks(mask_data, voxel_spacing)
        landmark_results[mask_name] = {
            "medial_point": medial_point,
            "lateral_point": lateral_point
        }
        
        # Format points for display with rounded values
        medial_point_str = f"({medial_point[0]:.2f}, {medial_point[1]:.2f}, {medial_point[2]:.2f})"
        lateral_point_str = f"({lateral_point[0]:.2f}, {lateral_point[1]:.2f}, {lateral_point[2]:.2f})"
        
        print(f"{mask_name:<20} {medial_point_str:<30} {lateral_point_str:<30}")
    
    print("-" * 80)
    print(f"All masks saved to '{output_dir}' directory and landmarks detected.")
    
    # Save landmark coordinates to a file for submission
    with open(os.path.join(output_dir, "landmark_coordinates.txt"), "w") as f:
        f.write("Tibia Landmark Coordinates (in mm)\n")
        f.write("-" * 80 + "\n")
        f.write(f"{'Mask Type':<20} {'Medial Point':<30} {'Lateral Point':<30}\n")
        f.write("-" * 80 + "\n")
        
        for mask_name, points in landmark_results.items():
            medial = points["medial_point"]
            lateral = points["lateral_point"]
            
            medial_str = f"({medial[0]:.2f}, {medial[1]:.2f}, {medial[2]:.2f})"
            lateral_str = f"({lateral[0]:.2f}, {lateral[1]:.2f}, {lateral[2]:.2f})"
            
            f.write(f"{mask_name:<20} {medial_str:<30} {lateral_str:<30}\n")
    
    print(f"Landmark coordinates saved to '{os.path.join(output_dir, 'landmark_coordinates.txt')}'")
    
    return landmark_results

if __name__ == "__main__":
    main()

Loading: segmented_tibia_random_2.0mm.nii.gz
Loaded tibia mask with shape: (512, 512, 216), voxel spacing: (0.869141, 0.869141, 2.0)
Generating masks...
Saving masks...
Saved: tibia_landmarks\original_mask.nii.gz
Saved: tibia_landmarks\expanded_2mm_mask.nii.gz
Saved: tibia_landmarks\expanded_4mm_mask.nii.gz
Saved: tibia_landmarks\randomized_mask_1.nii.gz
Saved: tibia_landmarks\randomized_mask_2.nii.gz
Detecting landmarks...

Landmark Coordinates (in mm):
--------------------------------------------------------------------------------
Mask Type            Medial Point                   Lateral Point                 
--------------------------------------------------------------------------------
Original Mask        (320.71, 238.14, 0.00)         (317.24, 242.49, 0.00)        
2mm Expanded Mask    (322.45, 238.14, 0.00)         (315.50, 242.49, 0.00)        
4mm Expanded Mask    (324.19, 238.14, 0.00)         (313.76, 242.49, 0.00)        
Randomized Mask 1    (321.58, 255.53, 0.00)    